In [10]:
import os
import mido
from mido import MidiFile
# from rich import print

In [11]:
def stretch_midi_file(file: MidiFile, new_duration_seconds):
    print(f"rescaling file from {file.length:.02f}s to {new_duration_seconds:.02f}s ({new_duration_seconds / file.length:.03f})")
    # Calculate stretch factor based on the original duration
    stretch_factor = new_duration_seconds / file.length
    
    # Scale the time attribute of each message by the stretch factor
    for track in file.tracks:
        for msg in track:
            msg.time = int(msg.time * stretch_factor)

    return file

In [20]:
data_dir = os.path.join("..", "inputs", "all-time")
target_bpm = 100

file =  os.listdir(data_dir)[0]
midi_file_path = os.path.join(data_dir, file)
midi_file = mido.MidiFile(midi_file_path)
bpm = float(os.path.basename(midi_file_path).split('-')[1])

print(f"found bpm {bpm} for file {os.path.basename(midi_file_path)}")
new_message = mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(target_bpm), time=0)
tempo_added = False
midi_file_cp = midi_file    

with mido.open_output('Disklavier') as output: # type: ignore
    for msg in midi_file.play(meta_messages=False):
        output.send(msg)
    
    print(len(midi_file_cp.tracks[0]))
    for track in midi_file_cp.tracks:
        # Remove existing set_tempo messages
        for j, msg in enumerate(track):
            if msg.type == 'set_tempo':
                print(f"removing set tempo message {msg}")
                _ = track.pop(j)
        
        # Add new set_tempo message to the first track
        if not tempo_added:
            print(f"adding message {new_message}")
            track.insert(0, new_message)
            tempo_added = True

    new_time = midi_file.length * bpm / target_bpm
    midi_file_cp = stretch_midi_file(midi_file_cp, new_time)

    if os.path.exists("tmp.mid"):
        os.remove("tmp.mid")
    midi_file_cp.save("tmp.mid")
    midi_file_cp = MidiFile("tmp.mid")

    # confirm that changes stuck
    midi_file_cp.print_tracks()
    for msg in midi_file_cp.play(meta_messages=False):
        output.send(msg)


found bpm 64.0 for file 20240124-064-02_0435-0442.mid
4
removing set tempo message MetaMessage('set_tempo', tempo=937500, time=0)
adding message MetaMessage('set_tempo', tempo=600000, time=0)
rescaling file from 7.50s to 4.80s (0.640)
=== Track 0
MetaMessage('set_tempo', tempo=600000, time=0)
MetaMessage('set_tempo', tempo=937499, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('end_of_track', time=0)
=== Track 1
MetaMessage('track_name', name='20240124-064-02_0435-0442', time=0)
Message('program_change', channel=0, program=0, time=0)
Message('note_on', channel=0, note=45, velocity=18, time=3)
Message('note_on', channel=0, note=61, velocity=60, time=0)
Message('note_on', channel=0, note=54, velocity=24, time=71)
Message('note_on', channel=0, note=54, velocity=0, time=19)
Message('note_on', channel=0, note=61, velocity=0, time=92)
Message('note_on', channel=0, note=63, velocity=52, time=0)
Message(